In [1]:
#%reset -f

import pandas as pd
import numpy as np
import scipy as sci
import matplotlib.pyplot as plt
import gurobipy as gp

from numpy import sqrt, exp, log, sin, cos, pi
from numpy.linalg import inv, pinv, det, matrix_rank, cond, lstsq, solve
from gurobipy import GRB, LinExpr
from numba import jit

plt.style.use('default')                                    # 使用默认风格
plt.rcParams['figure.facecolor'] = 'white'                  # 将图形的背景颜色设置为白色
plt.rcParams['font.family'] = 'YaHei Consolas Hybrid'       # 字体确认
plt.rcParams['font.size'] = 12                              # 字体大小
plt.rcParams['axes.unicode_minus'] = False                  # 正常显示负号
%matplotlib inline
# matplotlib.use('TkAgg')

lamb2baseKS = pd.read_excel('./第11题附件/KS与浓度关系.xlsx').iloc[0, 2:].values
coefDf = pd.read_excel('q1coef.xlsx')
stimulusDf = pd.read_excel('./第11题附件/光谱三刺激值加权表.xlsx')
q2targetDf = pd.read_excel('./第11题附件/问题二目标R值.xlsx')
coefDf.set_index('lamb', inplace=True)
coefDf.columns = ['redk','redb','yellowk','yellowb','bluek','blueb']
stimulusDf.set_index('lamb', inplace=True)

KeyError: "None of ['lamb'] are in the columns"

In [2]:
## 计算10个样品目标的XYZ值

targetX = []
targetY = []
targetZ = []

for i in range(10):
    X = 0
    Y = 0
    Z = 0
    for j in range(16):
        X += stimulusDf.iloc[j,0] * q2targetDf.iloc[i,j] * 20
        Y += stimulusDf.iloc[j,1] * q2targetDf.iloc[i,j] * 20
        Z += stimulusDf.iloc[j,2] * q2targetDf.iloc[i,j] * 20
    targetX.append(X * 0.1)
    targetY.append(Y * 0.1)
    targetZ.append(Z * 0.1)


In [ ]:
## 第二问优化模型
lamb = np.arange(400,701,20)
lamblen = len(lamb)

for i in range(10):

    ## 创建优化问题
    model1 = gp.Model('第二问优化模型')

    ## 创建优化变量
    c = model1.addVars(3, 1, lb=0, ub=5, vtype=GRB.CONTINUOUS, name='concentration')
    R = model1.addVars(lamblen, 1, vtype=GRB.CONTINUOUS, name='R')
    xyz = model1.addVars(3, 1, vtype=GRB.CONTINUOUS, name='xyz')

    ## 约束
    # K/S与R值的约束
    for j in range(lamblen):
        model1.addConstr(2 * R[j] * (lamb2baseKS[j] + 
                                    coefDf.redk[j] * c[0] + coefDf.redb[j] + 
                                    coefDf.yellowk[j] * c[1] + coefDf.yellowb[j] + 
                                    coefDf.bluek[j] * c[2] + coefDf.blueb[j]) == (1 - R[j])**2)

    # 求XYZ
    X = LinExpr()
    Y = LinExpr()
    Z = LinExpr()
    for j in range(lamblen):
        X += stimulusDf.iloc[j, 0] * R[j] * 20
        Y += stimulusDf.iloc[j, 1] * R[j] * 20
        Z += stimulusDf.iloc[j, 2] * R[j] * 20
    X *= 0.1
    Y *= 0.1
    Z *= 0.1
    
    # 色差计算
    
    
    ## 目标函数
    model1.setObjective()

